<a href="https://colab.research.google.com/github/davis689/binder/blob/master/CHEM461/Particle_on_a_ring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np # import numerical computation functions
import matplotlib.pyplot as plt # import plotting functions

#Particle on a Ring
We have already seen how we can numerically calculate the eigenfunctions and eigenvalues of particles in various types of boxes. These boxes have boundary conditions where at both ends of the box the eigenfunction either has to go to zero or the parts of the eigenfunctions inside and outside the box need to match up with each other at the boundary.

What would happen if we wrapped the one dimensional system around on itself and made the right-side of the box meet the left-side of the box forming a ring?

Here, as in the finite potential particle in a box, the wavefunction would need to be continuous such that at 0 and $2\pi$ the value of the wavefunction would match.



### Second order Differential
As [we saw previously](https://github.com/davis689/binder/blob/master/CHEM461/Numerical_solutions_to_Schrodinger_Equation.ipynb), a second derivative can be obtained with matrix multiplication using the matrix equation,
$$
\begin{pmatrix}f''_0 \\ f''_1 \\ f''_2 \\\vdots \\ f''_{N-1}\end{pmatrix} = \frac{1}{\Delta x^2}
\begin{pmatrix} -2 & 1 & 0 & 0 & \\ 1 & -2 & 1 & 0 & \\
0& 1 & -2 & 1 &  \\ &  & \ddots & \ddots & \ddots &\\
&  & & 1 & -2 \end{pmatrix}\begin{pmatrix}f_0 \\ f_1 \\ f_2 \\\vdots \\ f_{N-1}\end{pmatrix}=\begin{pmatrix}2f_0-f_1 \\ f_0-2f_1+f_2 \\ f_1-2f_2+f_3 \\\vdots \\ f_{N-2}-2f_{N-1}\end{pmatrix}
$$
The derivative matrix here has non-zero entries only on the diagonal and on the superdiagonal and subdiagonal one row above and below the diagonal.

In all cases so far, the first and last lines of this matrix are ignored because the second derivative method relies on evaluating the value of the function one point ahead and one point behind the position at which we are evaluating the derivative. Since there is no point behind the first point and no point ahead of the last point, the method fails at these points. That's OK because we just throw these points away and evaluate the derivative at the other $N$-2 points.

In the present case, we need to modify the derivative matrix to make clear that the point after the $(N-1)^{th}$ point (not the $N^{th}$ point since we're numbering starting with 0 which means the last point is point $N-1$) is the 0$^{th}$ point and the one before the $0^{th}$ point is the $(N-1)^{th}$ point. This means adding a 1 to the last entry in the first line of the derivative matrix as well as the last entry in the first column. Doing this means that multiplication of the first row with the eigenvector gives $f_{N-1}-2f_0+f_1$ and the last row multiplied by the eigenfunction gives $f_{N-2}-2f_{N-1}+f_0$.

# Solving the Particle in a Box Schrödinger Equation

We can now setup the Schrodinger Equation as a matrix equation:
$$
\hat H = \frac{\hbar^2}{2m}\frac{d^2}{d x^2} + V \\
\hat H \psi(x) = E \psi(x)
$$
We now know the matrix for taking the second order derivative. The matrix for the potential is simply the values of the potential on the diagonal of the matrix: $\mathbf{V}_{i=j} = V_i$. The result of multiplying the function vector will be to simply multiply each element of the function by the value of the potential.

Writing out the matrix for $\mathbf{H}$, including the two new off-diagonal elements to account for the connection between the sides of the box as we wrap it back on itself, we get:
$$
\mathbf{H} = \frac{-\hbar^2}{2 m \Delta x^2} \begin{pmatrix} -2 & 1 & 0 & 0 & &\mathbf{1}\\ 1 & -2 & 1 & 0 & &\\
0& 1 & -2 & 1 &  \\ &  & \ddots & \ddots & \ddots &&\\
\mathbf{1}&  && & 1 & -2 \end{pmatrix} +
\begin{pmatrix} V_0 & 0 & 0 & & \\ 0 & V_1 & 0 & & \\ 0 & 0 & V_2 & & \\ & & &\ddots & \\ &&&&V_{N-1}\end{pmatrix}
$$

It is worth looking at the matrix of the Hamiltonian and notice the symmetry: $\mathbf{H}^T = \mathbf{H}$, so the transpose of the matrix is identical to the matrix. Since the matrix is *real* everywhere, the complex conjugate is also the same: $\mathbf{H}^*=\mathbf{H}$. Combining these two statements, we can say that the Hamiltonian is Hermetian: $\mathbf{H}^\dagger = \mathbf{H}$.



In [4]:
hbar,m,r=1,1,1 # use atomic units

N = 1000
phi = np.linspace(0.,np.pi*2.,N)

Δphi = 2*np.pi/N # This is the difference between points.



Now we set up the potential energy, the second derivative operator, and the Hamiltonian. Then extract the eigenvalues and eigenfunctions.

In [5]:
V = 0.*phi # could be just zero but put x here to show that it is a function of x.
D2 = 1./(Δphi**2)*(np.diag(np.ones(N-1),-1) -2* np.diag(np.ones(N),0) + np.diag(np.ones(N-1),1)+np.diag(np.ones(1),N-1)+np.diag(np.ones(1),-N+1)) # derivative operator
H = -(hbar*hbar)/(2.0*m*r**2)*D2 + np.diag(V)  # Hamiltonian including derivative and potential energy.


E,psiT = np.linalg.eigh(H) # This computes the eigenvalues and eigenvectors
psi = np.transpose(psiT)   # We take the transpose of psiT to the wavefunction vectors can accessed as psi[n]

In [ ]:
D2

The energies are stored in $E$. The first $N$ levels are calculated. We will focus on only a few of them. Let's list the first several in atomic energy units.

In [ ]:
for i in range(20):
  print("E_{}={:8.3f}".format(i+1,E[i])) # the notation here is strange. Basically we will print everything between the ""
                                         #but stuff between the brackets will get subsitututed in from the between the parentheses in .format()
                                         #So here we'll print i+1 which is the quantum number 'n' in the particle in a box and the energies.
                                         #The :8.3f stuff indicates how many places to use for the whole number (some may be spaces) and how many for the
                                         #after the decimal point. The 'f' indicates a floating point (or decimal number). If we don't care about the
                                         #format of the number that is displayed or if the number is a whole number as it is, like 'i', just {} is fine.

####Scaling the energies
The energy of the lowest level in a particle in a box is $\frac{n^2 \hbar^2 \pi^2}{2 m a^2}$. For convenience we will rescale the energies to be relative to the energy of the first level. So $\epsilon=\frac{E_n}{E_1}=\frac{E_n}{\frac{\hbar^2 \pi^2}{2ma^2}}=\frac{E_n}{\frac{\pi^2}{2}}=\frac{2E_n}{\pi^2}$ and $ϵ=\frac{E_n}{\frac{\hbar^2 \pi^2}{2ma^2}}=\dfrac{\frac{n^2 \hbar^2 \pi^2}{2ma^2}}{\frac{\hbar^2 \pi^2}{2ma^2}}=n^2$. So we can rescale the energy by dividing by $\pi^2/2$ and the resulting value should be the quantum number squared. This makes it easy to see at a glance how close our numerical approximate answer is the analytical answer.

In [ ]:
ϵ=[E[i]/(hbar**2/(2*m*r**2)) for i in range(50)] # we only need divide by 1/2 but hbar, m, and r are set equal to 1 so...
for i in range(20):
  print("ϵ_{}={:>8.3f}".format(i+1,ϵ[i]))

Our calculation gives almost what we expect. Each energy is essentially $m^2$ as it should be. If you change $N$ above and re-run the cells, you can see what affect using fewer (or more) points will have on runtime and accuracy. Try N=100 and N=5000. Remember that our calculation uses an N by N matrix so N=5000 will have to deal with a 25,000,000 element matrix.

In [9]:
norm=[np.sqrt(1/np.trapz(np.conj(psi[i])*psi[i],phi)) for i in range(50)] # calculate normalization for each eigenfunction
psi_norm=[psi[i]*norm[i] for i in range(50)] # normalize psi

We'll set up a mechanism to plot a bunch of wavefunctions at the same time.

The sign of the wavefunctions as we approach the ends is arbitrary. We can make it consistent if we want. The $for$ loop below plots each wavefunction but it also makes it so that the left end points all approach zero from above. The right end points then alternate.

In [ ]:
plt.figure(figsize=(10,7))
fact=0.1 # scale for plotting
nl=9
for i in range(0,nl): # start with i=1 and go to i=
      plt.plot(phi,psi_norm[i]/np.sqrt(Δphi)*fact+ϵ[i],label="$ϵ_{}$={:>8.3f}".format(i,ϵ[i]))
plt.title("Eigenfunctions of the Particle on a Ring")
plt.legend()
plt.savefig("Infinite_Square_Well_WaveFunctions.pdf")
plt.show()

In [ ]:
plt.figure(figsize=(4,4))
r=1
ef=9
scale=.2
x=r*np.cos(phi)
y=r*np.sin(phi)
xpsi=(r+psi_norm[ef]*scale)*np.cos(phi)
ypsi=(r+psi_norm[ef]*scale)*np.sin(phi)
plt.plot(x,y)
plt.plot(xpsi,ypsi)
plt.show()

We now also want to check that the energy levels do indeed correspond to the known levels:
$$
E_n = \frac{m^2 \hbar^2}{2 m r^2}
$$

In [ ]:
energy = [0] + [x for i in range(1, 11) for x in [i, -i]]

for i in range(nl):
    n = i+1
    print("ϵ[{}] = {:9.4f},     ϵ_{} ={:9.4f},      %deviation={:>8.4f}%".format(n,ϵ[i],n, energy[i]**2,((ϵ[i]-energy[i]**2)/energy[i]**2)))

A final test shows the accuracy of our calculation in the orthonormality of the states:

In [ ]:
np.trapz(psi_norm[1]**2,x) # check that it works for one psi

The code above checks the normalization of the n=2 eigenfunctions. We can check the orthonormality of a range of levels using the following lines.

In [ ]:
for j in range(nl):
    for i in range(nl):
        print("ψ_{}, ψ_{}, {:2.2f}".format(i+1,j+1,np.trapz(psi_norm[i]*psi_norm[j],x)))

Explain the results in terms of orthonormality.

